In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp4/sample_submission.csv
/kaggle/input/2021-ml-tp4/class_info.csv
/kaggle/input/2021-ml-tp4/train_label.csv
/kaggle/input/2021-ml-tp4/test/0261.avi
/kaggle/input/2021-ml-tp4/test/0435.avi
/kaggle/input/2021-ml-tp4/test/0309.avi
/kaggle/input/2021-ml-tp4/test/0039.avi
/kaggle/input/2021-ml-tp4/test/0162.avi
/kaggle/input/2021-ml-tp4/test/0402.avi
/kaggle/input/2021-ml-tp4/test/0255.avi
/kaggle/input/2021-ml-tp4/test/0416.avi
/kaggle/input/2021-ml-tp4/test/0354.avi
/kaggle/input/2021-ml-tp4/test/0486.avi
/kaggle/input/2021-ml-tp4/test/0113.avi
/kaggle/input/2021-ml-tp4/test/0106.avi
/kaggle/input/2021-ml-tp4/test/0291.avi
/kaggle/input/2021-ml-tp4/test/0407.avi
/kaggle/input/2021-ml-tp4/test/0247.avi
/kaggle/input/2021-ml-tp4/test/0442.avi
/kaggle/input/2021-ml-tp4/test/0193.avi
/kaggle/input/2021-ml-tp4/test/0044.avi
/kaggle/input/2021-ml-tp4/test/0136.avi
/kaggle/input/2021-ml-tp4/test/0332.avi
/kaggle/input/2021-ml-tp4/test/0311.avi
/kaggle/input/2021-ml-tp4/tes

In [2]:
import os
import numpy as np
import pandas as pd
import tqdm
import time
import cv2
import pickle
import torch

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from scipy.stats import mode
from sklearn.decomposition import PCA


In [3]:
# 베이스라인 달성을 위한 파라미터 제공
arg_img_size = (128, 128)
arg_dense_sift = True
args_local_cluster = 200
args_global_cluster = 200
num_frame = 5
args_aggr = "vlad" # or "bow"
pca_vlad = 128

# 비디오 전처리 및 프레임 별 특징점(visual word) 추출 (Empty Module 1)

In [4]:
# train 비디오의 행동 분류 label read
root = "/kaggle/input/2021-ml-tp4/"
train_csv = os.path.join(root, "train_label.csv")
train_csv = pd.read_csv(train_csv)
train_csv_arr = np.asarray(train_csv)

# 데이터 셋에 존재하는 행동 분류 정보 read
classinfo = os.path.join(root, "class_info.csv")
classinfo = pd.read_csv(classinfo)
classinfo_arr = np.asarray(classinfo)


train_path = os.path.join(root, "train")
test_path = os.path.join(root, "test")

# train 비디오 경로
train_list = os.listdir(train_path)
train_list.sort()
train_list = [os.path.join(train_path, i) for i in train_list]

# test 비디오 경로
test_list = os.listdir(test_path)
test_list.sort()
test_list = [os.path.join(test_path, i) for i in test_list]

In [5]:
def video_to_frame(video_path, size, num_frame):
    
    #########################################################
    ## 비디오에서 프레임을 추출해주는 함수
    ## 
    ## Input 
    ##     video_path : 한 비디오의 경로
    ##     size : 비디오 내의 프레임을 읽을 때, 원하는 해상도 크기
    ##     num_frames : 한 비디오 내에서 읽을 프레임의 수
    ##
    ## Output
    ##     frames : 읽고 저장한 총 프레임
    #########################################################
    cap = cv2.VideoCapture(video_path)
    
    # 한 비디오의 총 프레임 수 반환 및 원하는 프레임 수 많큼 읽기 위해 읽을 프레임의 인덱스 설정
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sel_ind = np.linspace(0, total_frame-1, num_frame).astype("int")
    
    
    num=0
    frames = []
    for i in range(total_frame):
        
        # 읽을 프레임 인덱스의 경우 프레임 읽어 메모리에 저장, 아닐 경우 지나감
        if i in sel_ind:
            res, frame = cap.read()
            # 원하는 해상도로 조절 및 grayscale로 변환
            frame = cv2.resize(frame, size, interpolation = cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)
        else:
            res = cap.grab()        
    cap.release()
    frames = np.asarray(frames)

    return frames

In [6]:
def computeSIFT(data, dense=False):
    
    #########################################################
    ## 비디오 내의 프레임에서 특징점(visual word)을 SIFT or DenseSIFT로 추출해주는 함수
    ## 
    ## Input 
    ##     data : 한 비디오에서 읽고 저장한 프레임
    ##     dense : SIFT or DenseSIFT 사용 여부
    ##
    ## Output
    ##     x : 프레임에 대해 추출된 특징점(visual word), dict 형태 -> x[0]이면 0번째 인덱스 프레임의 특징점(visual word) [n,128] 확인 가능
    #########################################################
    
    x = {}
    for i in range(0, len(data)):
        if dense:
            img = data[i]
            step_size = 8
            print("!")
            kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
            
            ####### Empty Module 1 : DenseSIFT ########
            # 기본 SIFT 와 동일하게 정의 
            # 기본 SIFT 에서는 detectAndCompute를 사용했지만 
            # Dense SIFT는 위에서 생성한 keypoint를 사용해 compute 만을 진행 
            ###########################################
            
            #sift 생성은 아래 기본 sift일때와 동일하게 구성하였다.
            sift = cv2.SIFT_create()
            #img를 data[i]로 추출했기에 그 과정은 건너뛰고 sift.compute를 바로 적용한다.
            _, desc = sift.compute(img, kp)
            
            
        else:
            sift = cv2.SIFT_create()
            img = data[i]
            kp, desc = sift.detectAndCompute(img, None)
        x.update({i : desc})

    return x

In [7]:
# train 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 train_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
train_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(train_list, desc="Extract {} in train data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    train_local_desc.update({vid_path : local_desc})

# test 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 test_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
test_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(test_list, desc="Extract {} in test data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    test_local_desc.update({vid_path : local_desc})


Extract dsift in train data:   0%|          | 3/2020 [00:00<03:17, 10.22it/s]

!
!
!
!
!
!
!
!
!
!
!
!
!
!
!


Extract dsift in train data:   0%|          | 5/2020 [00:00<02:40, 12.59it/s]

!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!


Extract dsift in train data:   0%|          | 9/2020 [00:00<02:20, 14.34it/s]

...

!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!


Extract dsift in test data:  99%|█████████▉| 502/505 [00:32<00:00, 15.59it/s]

!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!


Extract dsift in test data: 100%|██████████| 505/505 [00:32<00:00, 15.45it/s]

!
!
!
!
!
!
!
!
!
!
!


In [8]:
print("\n\nAggregate SIFT descriptor")
start = time.time()


# train 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 train_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
train_frame_total = []
train_local_desc_total = []
train_local_info = {}
for k, v in train_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            train_local_desc_total.extend(vv)
            train_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        train_local_info.update({k+", "+str(kk) : l_num})
train_local_desc_total = np.asarray(train_local_desc_total)
train_frame_total = np.asarray(train_frame_total)


# test 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 test_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 test_local_info에서 확인 가능
test_frame_total = []
test_local_desc_total = []
test_local_info = {}
for k, v in test_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            test_local_desc_total.extend(vv)
            test_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        test_local_info.update({k+", "+str(kk) : l_num})
test_local_desc_total = np.asarray(test_local_desc_total)
test_frame_total = np.asarray(test_frame_total)


print("\t{:3.2f}s\n\n".format(time.time()-start))



Aggregate SIFT descriptor
	3.37s




# 비디오 feature 기술을 위한 프레임 feature 기술 (Empty Module 2,3,4)

In [9]:
def clustering(train_desc, test_desc=None, n_clusters=200):
    #########################################################
    ## 모든 특징점들 중, 대표 특징점(codebook)을 선정하는 함수
    ## 
    ## Input 
    ##     train_desc : 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     test_desc : 모든 test 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     n_clusters : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     train_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     test_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     clusters : 대표 특징점(codebook)
    ##     kmeans : kmeans 인스턴스
    #########################################################
    
    
    
    ##### Empty Module 2 : 대표 특징점(codebook) 선정 ######
    # 제약 조건 : MiniBatchKMeans 사용, random_state=0 고정
    # sklearn의 MiniBatchKMeans를 활용하여 n_clusters 크기 만큼의 대표 특징점(codebook)을 선정
    ###########################################
    
    #MiniBatchKMeans는 가이드 라인대로 생성한 후 train_desc로 학습하고 예측데이터를 만들었다.
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(train_desc)

    train_pred = kmeans.predict(train_desc)
    if test_desc is not None:
        test_pred = kmeans.predict(test_desc)
    else:
        test_pred = None
    
    #kmeans가 속성값으로 가지는 cluster_centers_를 이용해서 대표점들을 뽑았다.
    clusters = kmeans.cluster_centers_
    
    return train_pred, test_pred, clusters, kmeans

In [10]:
# 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들로 대표 특징점(codebook)을 만들고,
# train 비디오의 모든 프레임에서 추출된 특징점(visual word)과 test 비디오의 모든 프레임에서 추출된 특징점(visual word)을 대표 특징점(codebook)에 할당
train_local_alloc, test_local_alloc, local_codebook, local_kmeans = clustering(train_local_desc_total, test_local_desc_total, args_local_cluster)

In [11]:
def VLAD(X, alloc, centers):
    #########################################################
    ## 이미지 feature인 VLAD feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     X : 한 프레임의 특징점(visual word)들
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     centers : 대표 특징점(codebook)
    ##
    ## Output
    ##     V : VLAD feature
    #########################################################
    
    m,d = X.shape
    k = centers.shape[0]
    
    # VLAD feature를 담기 위한 변수
    V=np.zeros([k,d])
    for i in range(k):
        if np.sum(alloc==i)>0:
            ####################### Empty Module 3 : VLAD ########################
            # 이미지에서 추출된 특징점(visual word) X와 이들이 대표 특징점(codebook)으로 할당된 정보 alloc을 이용해,
            # 동일한 대표 특징점(codebook)으로 할당된 특징점(visual word)들의 벡터 합 계산해 V[i]에 저장
            # hint : 바로 위 조건문의 조건 "alloc==i"의 의미를 파악하고 인덱싱에 활용
            ######################################################################
            
            #오버뷰를 확인해보면 이제 대표 중심점에 할당된 즉 대표 중심점의 군집에 해당하는 특징점들과
            #대표 특징점 사이의 벡터 차이값을 더하여 넣는 과정이다.
            #X[alloc == i]를 하게 되면, 대표 특징점들의 위치정보를 담고있는 alloc에서
            #i와 같은 즉 각 대표 특징점에 해당하는 X의 값들을 추출하여 볼 수 있게 된다.
            #이렇게 추출된 특징점들에서 centers[i] 즉 대표 특징점을 빼준 후 axis=0를 이용해서 np.sum을 하여
            #차이점을 합해서 V[i,:]에 저장해준다.
            V[i, :] = np.sum(X[alloc==i, :]-centers[i], axis = 0)
            
    
    # 후처리 과정
    V = V.flatten()
    V = np.sign(V)*np.sqrt(np.abs(V))
    if np.sqrt(np.dot(V,V))!=0:
        V = V/np.sqrt(np.dot(V,V))
    return V


def BoW(alloc, n_cluster):
    #########################################################
    ## 이미지 feature인 BoW feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     n_cluster : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     V : BoW feature
    #########################################################
    
    ######################### Empty Module 4 : BoW ##########################
    # 이미지에서 추출된 특징점(visual word)이 대표 특징점(codebook)으로 할당된 정보 alloc의 histogram을 계산
    # np.histogram 함수 참고
    #########################################################################
    
    #해당 부분은 11주차 코드를 많이 참고하였다.
    #11주차에서 index를 나타내던 부분이 이제 위치정보이기에 alloc을 담고
    #갯수를 담는 부분에서 n_cluster를 대신하여 넣어준 뒤 나머지는 동일하게 Bow함수를 구성하였다.
    V = np.histogram(alloc, bins=range(n_cluster + 1), normed=True)[0].flatten()
    
    return V

In [12]:
print("\n\nAllocate center & Descript local histogram")
start = time.time()

# Train 비디오 내의 프레임 별로 이미지 feature 기술 -> train_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> train_global_desc_key
train_global_desc = []
train_global_desc_key = []
vi=0
for k, v in train_local_info.items():
    if v!=0:
        if args_aggr=="bow":            
            hist_desc = BoW(train_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(train_local_desc_total[vi:vi+v], train_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()
        vi+=v
        train_global_desc.append(hist_desc)
        train_global_desc_key.append(k)
train_global_desc = np.asarray(train_global_desc)
train_global_desc_key = np.asarray(train_global_desc_key)


# Test 비디오 내의 프레임 별로 이미지 feature 기술 -> test_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> test_global_desc_key
test_global_desc = []
test_global_desc_key = []
vi=0
for k, v in test_local_info.items():
    if v!=0:
        if args_aggr=="bow":
            hist_desc = BoW(test_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(test_local_desc_total[vi:vi+v], test_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        test_global_desc.append(hist_desc)
        test_global_desc_key.append(k)
test_global_desc = np.asarray(test_global_desc)
test_global_desc_key = np.asarray(test_global_desc_key)

print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript local histogram
	63.10s




In [13]:
# VLAD feature의 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_global_desc)
    train_global_desc = pca.transform(train_global_desc)
    test_global_desc = pca.transform(test_global_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))



Reduce dim of descriptor of the frames with PCA
	32.74s




In [14]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 train 비디오의 각 프레임 별 label 가공
train_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_global_desc_key])
train_global_label = []
for fid in train_global_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_global_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_global_label = np.asarray(train_global_label).ravel()

# 분류를 위해, 행동 분류에 대한 test 비디오의 각 프레임 별 id 가공 
test_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_global_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.49s




In [15]:
def saveFile(predict, predict_id, name, best_params=None):
    #########################################################
    ## 결과를 저장하기 위한 함수
    ## 
    ## Input 
    ##     predict : 모든 test 비디오의 행동 예측 값
    ##     predict_id : 모든 test 비디오의 id
    ##     name : 원하는 저장 파일 이름
    ##     best_params : 원하는 instance 저장 시 사용
    ##
    #########################################################
    
    data = np.concatenate((np.expand_dims(predict_id.astype("str"), axis=1), np.expand_dims(predict.astype("str"), axis=1)), axis=1)
    csv = pd.DataFrame(data, columns=['Id', 'Category'])
    csv.to_csv(name + ".csv", index=False)
    
    if best_params:
        f = open(name + ".pickle", "wb")
        pickle.dump(best_params, f, 2)

# 비디오의 모든 프레임에서 얻은 feature를 평균내어 비디오 feature 생성 및 분류 (Empty Module 5)

In [16]:
print("\n\nSVM global averaging in frame")
start = time.time()
#################### Empty Module 6 : SVM (averaging) ######################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 비디오 별로 평균 내어 비디오 feature로 사용
# 비디오 feature로 학습 후, 행동 예측
# hint : 위 셀에서 선언한 train_global_id, train_global_label, test_global_id 활용
###########################################################################
from sklearn.model_selection import GridSearchCV
#params = {'gamma': ['auto', 'scale'], 'tol':[0.01, 0.05, 0.1, 0.5]}
#svm = GridSearchCV(SVC(random_state=0, C=3), params, cv=10, n_jobs=-1)

#해당 부분에서 학습 데이터에 대해 찍어본 결과 C가 4부터 
#학습데이터에 대한 점수가 1.0이 나오는 것을 확인하였다.
#그래서 4에서 낮추며 확인을 해보았는데, 3에서 
#가장 높은 결과값이 나와 3으로 둔 후 tol과 gamma를 조정하면서
#그리드 서치를 굴려보았다.
svm = SVC(random_state=0, C=3, tol=0.5)

#평균값을 낼때, 일단 각 데이터의 갯수를 체크하였다.
trs = train_global_id.shape[0]
tes = test_global_id.shape[0]

#새로운 빈 데이터를 만든 후
train_global_avg = []
test_global_avg = []
train_global_label_avg = []

#각 갯수만큼 5개씩 돌면서 평균을 구해 더하고 추가해주었다.
#label은 5개마다 동일했기에 따로 평균을 구하지는 않고 그저 append로 추가하였다.
for i in range(0, trs, 5):
    train_global_avg.append(np.mean(train_global_desc[i : i + 5, :], axis=0))
    train_global_label_avg.append(train_global_label[i])

#test에 대해서도 동일하게 구성하였다.
for i in range(0, tes, 5):
    test_global_avg.append(np.mean(test_global_desc[i : i + 5, :], axis=0))

#추가한 데이터들이 리스트이기에 이를 행렬로 바꾸어주기 위해서 np.array함수를 이용했다.
train_global_avg = np.array(train_global_avg)
train_global_label_avg = np.array(train_global_label_avg)
test_global_avg = np.array(test_global_avg)

#찾은 파라미터로 원래 데이터로 학습을 시켜보았다.
svm.fit(train_global_avg, train_global_label_avg)

#원래 데이터에 대해서 학습데이터에 대한 점수를 보기 위해 점수를 출력해보았다.
print(svm.score(train_global_avg, train_global_label_avg))

svm_predict = svm.predict(test_global_avg)
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_averaging")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global averaging in frame
0.9985148514851485
	2.04s




# 비디오의 모든 프레임에서 얻은 feature로 분류기를 사용해 예측하고 예측치 중 가장 많은 빈도를 나타낸 행동을 선정 (Empty Module 6)

In [17]:
print("\n\nSVM global voting in frame")
start = time.time()
############################### Empty Module 5 : SVM (voting) ##################################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 모두 사용해 SVM 학습
# 학습 시, 각 이미지 feature의 label은 해당되는 비디오의 label(행동)로 사용
# 프레임 별로 행동 예측 후, 같은 비디오 내 프레임의 행동 예측 값의 최빈값을 해당 비디오의 행동 예측 값으로 선정
# hint : 위 셀에서 선언한 train_global_label, test_global_id 및 mode 활용
################################################################################################

#GridSearchCV를 이용해서 최적의 값을 찾아보았다. 결론적으로는 voting보다 average가 더 낮게 나오기에
#추가적인 파라미터 조정을 하지 않았다.
from sklearn.model_selection import GridSearchCV
params = {'C':[0.01, 0.1, 10, 100, 1000]}
svm = GridSearchCV(SVC(random_state=0), params, n_jobs=-1)
svm.fit(train_global_desc, train_global_label)
print(svm.best_params_)
svm = svm.best_estimator_

#전체에 대한 데이터의 결과값을 일단 담아두었다.
svm_predict_before = svm.predict(test_global_desc)

svm_predict = []

#테스트 전체 갯수에 대해서 5개씩 돌면서 5개마다 가장 많이 나온 예측값을 담아서 만들었다.
for i in range(0, tes, 5):
    svm_predict.append(mode(svm_predict_before[i : i + 5]).mode[0])
    
#만든 데이터를 넣을 수 있게 np.array함수를 이용했다.
svm_predict = np.array(svm_predict)

saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_voting")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global voting in frame
{'C': 10}
	177.57s




# 프레임 feature에서 대표되는 feature를 선정 후 BoW or VLAD 방식으로 비디오 feature를 기술 (Empty Module 7)

In [18]:
train_global_alloc, test_global_alloc, global_codebook, global_kmeans = clustering(train_global_desc, test_global_desc, args_global_cluster)

In [19]:
print("\n\nAllocate center & Descript global histogram")
start = time.time()
train_vid_names = np.asarray([i.split(", ")[0] for i in train_global_desc_key])
train_vid_names_u = np.unique(train_vid_names)

# Train 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
train_video_desc = []
train_video_desc_key = []
for vid_name in train_vid_names_u:
    cind = np.where(vid_name==train_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(train_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(train_global_desc[cind], train_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    train_video_desc.append(hist_desc)
    train_video_desc_key.append(vid_name)
train_video_desc = np.asarray(train_video_desc)
train_video_desc_key = np.asarray(train_video_desc_key)

# Test 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
test_vid_names = np.asarray([i.split(", ")[0] for i in test_global_desc_key])
test_vid_names_u = np.unique(test_vid_names)

test_video_desc = []
test_video_desc_key = []
for vid_name in test_vid_names_u:
    cind = np.where(vid_name==test_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(test_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(test_global_desc[cind], test_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    test_video_desc.append(hist_desc)
    test_video_desc_key.append(vid_name)
test_video_desc = np.asarray(test_video_desc)
test_video_desc_key = np.asarray(test_video_desc_key)


print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript global histogram
	9.94s




In [20]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 각 train 비디오 별 label 가공
train_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_video_desc_key])
train_video_label = []
for fid in train_video_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_video_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_video_label = np.asarray(train_video_label).ravel()

# 분류를 위해, 행동 분류에 대한 각 test 비디오 별 id 가공
test_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_video_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))



Processing label
	0.13s




In [21]:
# 이미지 feature에 대해 다시 한번 VLAD feature 기술 방식을 사용하여 video feature를 기술한 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_video_desc)
    train_video_desc = pca.transform(train_video_desc)
    test_video_desc = pca.transform(test_video_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))




Reduce dim of descriptor of the frames with PCA
	7.20s




In [22]:
print("\n\nSVM video descriptor")
start = time.time()
######################## Empty Module 7 : SVM (video feature) ##########################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 이전 이미지 feature를 기반으로 각 기술방식(BoW, VLAD)을 한번 더 적용해 만든 비디오 feature 사용
# 비디오 feature로 학습 후, 행동 예측
#######################################################################################
#기본적으로 주어진 데이터를 이용해서 학습을 진행했다.
#성능이 다른 항목들에 비해서 많이 떨어지게 나와 추가적인 파라미터 조정은 진행하지 않았다.
from sklearn.model_selection import GridSearchCV
params = {'C':[0.01, 0.1, 1, 10, 100, 1000]}
svm = GridSearchCV(SVC(random_state=0, class_weight='balanced'), params)
svm.fit(train_video_desc, train_video_label)

svm_predict = svm.predict(test_video_desc)

saveFile(classinfo_arr[svm_predict][:,1], test_video_id, "svm_video")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM video descriptor
	24.62s


